<h1> FOOFing OMEGA Data </h1>

This notebook runs FOOF on MEG-PSD data.
- Option to Parallelize with Pool from Multiprocessing
- Must Specify frequency range desired for FOOF
- Saves FOOF results in pickled object

NOTES:
- For reasons unknown, parallel implementation does not currently work. 

    -It used to, but now hangs when trying to parallelize foof. 

In [10]:
# Import sys and add FOOF location to path
import sys
sys.path.append('/Users/thomasdonoghue/Documents/GitCode/')

# Import FOOF
from foof import syn
from foof.fit import FOOF

# Import custom code for running MEG-FOOF
from om_foof import *

# Import Parallelization Packages
from multiprocessing import Pool, freeze_support

# Import other packages
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Set plots inline
%matplotlib inline

In [11]:
# Set Paths to MEG Data
meg_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/2-PSDs/Colin/'

In [12]:
# Check Availabe Subjects
files = os.listdir(meg_path)
files = clean_file_list(files, 'Subject_')
print "Number of Subjects", len(files)
print files

Number of Subjects 21
['Subject_111802', 'Subject_124340', 'Subject_218131', 'Subject_220216', 'Subject_245518', 'Subject_265922', 'Subject_320548', 'Subject_339964', 'Subject_369737', 'Subject_386068', 'Subject_390845', 'Subject_403435', 'Subject_405716', 'Subject_431088', 'Subject_504997', 'Subject_548027', 'Subject_604600', 'Subject_704718', 'Subject_704814', 'Subject_756575', 'Subject_978341']


In [13]:
# Load MEG Data
subj = 111802
psd, freqs = load_meg_dat(meg_path, subj)

In [14]:
# Check data - get number of PSDs and frequency resolution
[nPSDs, nFreqs] = np.shape(psd)
freq_res = np.mean(np.diff(freqs))
print 'Number of PSDs is: ', nPSDs
print 'Frequency Resolution is: ', freq_res

Number of PSDs is:  7500
Frequency Resolution is:  0.48828125


In [15]:
# Extract PSD range of interest
psd_ext, freqs_ext = extract_psd(psd, freqs, f_low=3, f_high=40)

In [17]:
# Run FOOF
foof_results = meg_foof(psd_ext[1:10, :], freqs_ext, method='linear', min_p=0.1, freq_res=freq_res)

# Save out results
#save_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/'
#save_pickle(foof_results, save_path, subj)

In [ ]:
# Open Pool, run FOOF in parallel, and then close pool. 
# Results stored in list of tuples"
pool = Pool(8)
results = pool.map(foof_par, psd_list[0:1])
pool.close()
pool.join()

# Use pickle to save the results
foof_data_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/'
foof_save_name = str(subj) + '_Foof_Vertex.p'
foof_save_path = os.path.join(foof_data_path, foof_save_name)
pickle.dump(results, open(foof_save_path, 'wb'))

# Close the pool
pool.terminate()

In [ ]:
# Parallel not working. Run linearly. 
results = [foof_par(psd) for psd in psd_list]

# Use pickle to save the results
foof_data_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/'
foof_save_name = str(subj) + '_Foof_Vertex.p'
foof_save_path = os.path.join(foof_data_path, foof_save_name)
pickle.dump(results, open(foof_save_path, 'wb'))

In [ ]:
# EXTRA: CHECK FOOF

print("model: f {0}, f_sig {1}, chi {2}".format(
        np.round(foof.centers_, decimals=1),
        np.round(foof.stdevs_, decimals=1),
        np.round(foof.chi_, decimals=1))
     )

plt.subplot(211)
plt.plot(freqs_ext, psd_example[:,0]*10000000000000000000000, label='Data')
plt.plot(*syn.mfonef(mk=foof.powers_, 
                     mf=foof.centers_, 
                     mf_sig=foof.stdevs_, 
                     f0=3, 
                     fmax=50, 
                     chi=foof.chi_, 
                     res=freq_res), 
         label='Model'
        )
plt.xlabel("F (Hz)")
plt.ylabel("PSD")
plt.yscale('log')
plt.xscale('log')
plt.legend()